In [1]:
import bql
import numpy as np
from IPython.display import display, clear_output
import pandas as pd
import time 
import datetime
import bqviz as bqv
from scipy.optimize import minimize
from dateutil.relativedelta import relativedelta
import bqport
from bqport import PositionType
from sklearn import linear_model

bq = bql.Service()
d = bq.data
f = bq.func
u = bq.univ

equity_idx = ['ACWI US Equity', 'EFA US Equity', 'XMEU GY Equity', 'VDEV LN Equity', 'IWVL LN Equity','IEMG US Equity','XRSU LN Equity']
fi_idx = ['FLOA LN Equity', 'IGIL LN Equity', 'CBU7 LN Equity']

etf_list = equity_idx + fi_idx

In [2]:
# INPUTS ###############

index_to_replicate = 'HFRIFOF Index'

start_date = datetime.datetime(2006,1,1)
end_date = datetime.datetime(2019,1,1)
curncy = 'USD'
rotation = 1 #months

In [3]:
#The fund inception of some ETFs it's too close to crrent days, so to make the analysis we will asume that these ETFs replicate exactly the underlying indexes.
#Due to this hypothesis, the time series are going to be analyzed with the underlying indexes.

etf_under_idx = d.fund_benchmark()

request = bql.Request(u.list(etf_list), {'UND_IDX': etf_under_idx})
res = bq.execute(request)
etf_under_idx_list = [undr +' Index' for undr in list(res[0].df()['UND_IDX'])]

#For the time period specified, the underlying indexes of VDEV LN Equity, ACIM US Equity and XRSU LN Equity there aren't "longer enough (they start later)
#We asume that the Tracking Error between these indexes and the original is quite low

for i in range(len(etf_under_idx_list)):
    if etf_under_idx_list[i] == 'AWNT04U Index':
        etf_under_idx_list[i] = 'FTAW01 Index'
    elif etf_under_idx_list[i] == 'M1WDIM Index':
        etf_under_idx_list[i] = 'MXWDIM Index'
    elif etf_under_idx_list[i] == 'RU20N30U Index':
        etf_under_idx_list[i] = 'RTY Index'

In [4]:
#Price Aligneation ####################

#Get the whole price series por all the indexes of the ETFs
download_start_date = start_date - relativedelta(months=12)
download_end_date = end_date + relativedelta(months=rotation)

price = f.dropna(d.px_last(start = download_start_date, end = download_end_date))
request = bql.Request(u.list(etf_under_idx_list), {'Price': price})
res = bq.execute(request)
price_series = res[0].df().reset_index()

#Pivot the price series to get the df in a way that the rows are dates and columns indexes prices
align_price_series = price_series.pivot(index='DATE', columns='ID', values='Price').reset_index().dropna().reset_index().drop(['index'], axis=1)

#Add to the price series df an auxiliar date 
year_list = pd.DatetimeIndex(align_price_series['DATE']).year.tolist()
month_list = pd.DatetimeIndex(align_price_series['DATE']).month.tolist()
date_list = []
for j in range(len(year_list)):
    date_list += [str(year_list[j]) + '-' + str(month_list[j])]
align_price_series['aux_date'] = date_list  

#Get the price series for HFRIFOF Index (the returned data is in a monthly basis)
price = f.dropna(d.px_last(start = download_start_date, end = download_end_date))
request = bql.Request(index_to_replicate, {'Price': price})
res = bq.execute(request)
index_price_series = res[0].df().reset_index()

#Add to the index price series df an auxiliar date and generate the final df, where the index is the date and columns belong to the porfolio and HFRIFOF value
index_price_series = index_price_series.drop(['ID', 'CURRENCY'], axis=1)
index_price_series = index_price_series.rename(index=str, columns={'Price': index_to_replicate})
year_list = pd.DatetimeIndex(index_price_series['DATE']).year.tolist()
month_list = pd.DatetimeIndex(index_price_series['DATE']).month.tolist()
date_list = []
for j in range(len(year_list)):
    date_list += [str(year_list[j]) + '-' + str(month_list[j])]
index_price_series['aux_date'] = date_list

grouped_price_series = pd.DataFrame(data = {})

list_aux_dates = align_price_series['aux_date'].unique().tolist()
for i_date in list_aux_dates:
    aux_mat = align_price_series.loc[align_price_series['aux_date'].isin([i_date]), align_price_series.columns != 'DATE']
    i_idx = aux_mat.index.tolist()
    aux_mat = aux_mat.loc[i_idx[-1]].to_frame().T
    
    if i_date == list_aux_dates[0]:
        grouped_price_series = aux_mat
        
    else:
        grouped_price_series = grouped_price_series.append(aux_mat, ignore_index=True)

grouped_price_series = pd.merge(index_price_series, grouped_price_series, how = 'left', on=['aux_date', 'aux_date'])

In [5]:
#MDA: MEAN DECREASE ACCURACY

from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import r2_score
from collections import defaultdict
 
names = etf_under_idx_list
    
# X = (grouped_price_series[etf_under_idx_list]/grouped_price_series.loc[0, etf_under_idx_list]).as_matrix()
# Y = (grouped_price_series[index_to_replicate]/grouped_price_series.loc[0, index_to_replicate]).as_matrix()

X = grouped_price_series[etf_under_idx_list].pct_change().fillna(0).as_matrix()
Y = grouped_price_series[index_to_replicate].pct_change().fillna(0).as_matrix()

 
rf = RandomForestRegressor()
scores = defaultdict(list)
 
#crossvalidate the scores on a number of different random splits of the data
for train_idx, test_idx in ShuffleSplit(len(X), 1000, .3):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = Y[train_idx], Y[test_idx]
    r = rf.fit(X_train, Y_train)
    acc = r2_score(Y_test, rf.predict(X_test))
    for i in range(X.shape[1]):
        X_t = X_test.copy()
        np.random.shuffle(X_t[:, i])
        shuff_acc = r2_score(Y_test, rf.predict(X_t))
        scores[names[i]].append((acc-shuff_acc)/acc)
        
print("Features sorted by their score:")
print(sorted([(round(np.mean(score), 4), feat) for feat, score in scores.items()], reverse=True))

C:\blp\BQuant\environments\bqnt-1.17.1\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Features sorted by their score:
[(0.1399, 'MIMUEMRN Index'), (0.1016, 'NDUEACWF Index'), (0.098500000000000004, 'FTAW01 Index'), (0.048300000000000003, 'BFU5TRUU Index'), (0.037100000000000001, 'IDCOT3TR Index'), (0.032800000000000003, 'NDDUEAFE Index'), (0.0091999999999999998, 'M1WOEV Index'), (0.00069999999999999999, 'RTY Index'), (-0.0038999999999999998, 'NDDUE15 Index'), (-0.019900000000000001, 'BCIW1A Index')]


In [6]:
#MDA: MEAN DECREASE ACCURACY

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import r2_score
from collections import defaultdict
 
names = etf_under_idx_list
    
# X = (grouped_price_series[etf_under_idx_list]/grouped_price_series.loc[0, etf_under_idx_list]).as_matrix()
# Y = (grouped_price_series[index_to_replicate]/grouped_price_series.loc[0, index_to_replicate]).as_matrix()

X = grouped_price_series[etf_under_idx_list].pct_change().fillna(0).as_matrix()
Y = grouped_price_series[index_to_replicate].pct_change().fillna(0).as_matrix()
 
rf = GradientBoostingRegressor()
scores = defaultdict(list)
 
#crossvalidate the scores on a number of different random splits of the data
for train_idx, test_idx in ShuffleSplit(len(X), 1000, .3):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = Y[train_idx], Y[test_idx]
    r = rf.fit(X_train, Y_train)
    acc = r2_score(Y_test, rf.predict(X_test))
    for i in range(X.shape[1]):
        X_t = X_test.copy()
        np.random.shuffle(X_t[:, i])
        shuff_acc = r2_score(Y_test, rf.predict(X_t))
        scores[names[i]].append((acc-shuff_acc)/acc)
        
print("Features sorted by their score:")
print(sorted([(round(np.mean(score), 4), feat) for feat, score in scores.items()], reverse=True))

Features sorted by their score:
[(0.1605, 'MIMUEMRN Index'), (0.1033, 'NDUEACWF Index'), (0.087099999999999997, 'FTAW01 Index'), (0.082500000000000004, 'BFU5TRUU Index'), (0.062300000000000001, 'IDCOT3TR Index'), (0.054300000000000001, 'NDDUEAFE Index'), (0.0281, 'RTY Index'), (0.0038, 'M1WOEV Index'), (0.0011999999999999999, 'NDDUE15 Index'), (-0.0134, 'BCIW1A Index')]


In [7]:
#MDA: MEAN DECREASE ACCURACY

from sklearn.ensemble import AdaBoostRegressor
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import r2_score
from collections import defaultdict
 
names = etf_under_idx_list
    
# X = (grouped_price_series[etf_under_idx_list]/grouped_price_series.loc[0, etf_under_idx_list]).as_matrix()
# Y = (grouped_price_series[index_to_replicate]/grouped_price_series.loc[0, index_to_replicate]).as_matrix()

X = grouped_price_series[etf_under_idx_list].pct_change().fillna(0).as_matrix()
Y = grouped_price_series[index_to_replicate].pct_change().fillna(0).as_matrix()
 
rf = AdaBoostRegressor()
scores = defaultdict(list)
 
#crossvalidate the scores on a number of different random splits of the data
for train_idx, test_idx in ShuffleSplit(len(X), 1000, .3):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = Y[train_idx], Y[test_idx]
    r = rf.fit(X_train, Y_train)
    acc = r2_score(Y_test, rf.predict(X_test))
    for i in range(X.shape[1]):
        X_t = X_test.copy()
        np.random.shuffle(X_t[:, i])
        shuff_acc = r2_score(Y_test, rf.predict(X_t))
        scores[names[i]].append((acc-shuff_acc)/acc)
        
print("Features sorted by their score:")
print(sorted([(round(np.mean(score), 4), feat) for feat, score in scores.items()], reverse=True))

Features sorted by their score:
[(0.28520000000000001, 'MIMUEMRN Index'), (0.063500000000000001, 'NDUEACWF Index'), (0.0596, 'IDCOT3TR Index'), (0.044699999999999997, 'FTAW01 Index'), (0.0395, 'BFU5TRUU Index'), (0.0339, 'M1WOEV Index'), (0.0212, 'NDDUEAFE Index'), (0.0033, 'NDDUE15 Index'), (0.0023999999999999998, 'RTY Index'), (-0.0088999999999999999, 'BCIW1A Index')]


In [8]:
#MDA: MEAN DECREASE ACCURACY

from sklearn.neural_network import MLPRegressor
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import r2_score
from collections import defaultdict
 
names = etf_under_idx_list
    
# X = (grouped_price_series[etf_under_idx_list]/grouped_price_series.loc[0, etf_under_idx_list]).as_matrix()
# Y = (grouped_price_series[index_to_replicate]/grouped_price_series.loc[0, index_to_replicate]).as_matrix()

X = grouped_price_series[etf_under_idx_list].pct_change().fillna(0).as_matrix()
Y = grouped_price_series[index_to_replicate].pct_change().fillna(0).as_matrix()

rf = MLPRegressor(activation = 'logistic', solver='sgd', random_state=1)
scores = defaultdict(list)
 
#crossvalidate the scores on a number of different random splits of the data
for train_idx, test_idx in ShuffleSplit(len(X), 1000, .3):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = Y[train_idx], Y[test_idx]
    r = rf.fit(X_train, Y_train)
    acc = r2_score(Y_test, rf.predict(X_test))
    for i in range(X.shape[1]):
        X_t = X_test.copy()
        np.random.shuffle(X_t[:, i])
        shuff_acc = r2_score(Y_test, rf.predict(X_t))
        scores[names[i]].append((acc-shuff_acc)/acc)
        
print("Features sorted by their score:")
print(sorted([(round(np.mean(score), 4), feat) for feat, score in scores.items()], reverse=True))

Features sorted by their score:
[(0.014200000000000001, 'M1WOEV Index'), (0.010200000000000001, 'NDUEACWF Index'), (0.0064999999999999997, 'NDDUE15 Index'), (0.0057000000000000002, 'FTAW01 Index'), (0.00040000000000000002, 'BCIW1A Index'), (-0.0, 'BFU5TRUU Index'), (-0.00020000000000000001, 'IDCOT3TR Index'), (-0.0020999999999999999, 'RTY Index'), (-0.0035999999999999999, 'MIMUEMRN Index'), (-0.024299999999999999, 'NDDUEAFE Index')]


In [9]:
#MDA: MEAN DECREASE ACCURACY

from sklearn.svm import SVR
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import r2_score
from collections import defaultdict
 
names = etf_under_idx_list
    
# X = (grouped_price_series[etf_under_idx_list]/grouped_price_series.loc[0, etf_under_idx_list]).as_matrix()
# Y = (grouped_price_series[index_to_replicate]/grouped_price_series.loc[0, index_to_replicate]).as_matrix()

X = grouped_price_series[etf_under_idx_list].pct_change().fillna(0).as_matrix()
Y = grouped_price_series[index_to_replicate].pct_change().fillna(0).as_matrix()
 
rf = SVR()
scores = defaultdict(list)
 
#crossvalidate the scores on a number of different random splits of the data
for train_idx, test_idx in ShuffleSplit(len(X), 1000, .3):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = Y[train_idx], Y[test_idx]
    r = rf.fit(X_train, Y_train)
    acc = r2_score(Y_test, rf.predict(X_test))
    for i in range(X.shape[1]):
        X_t = X_test.copy()
        np.random.shuffle(X_t[:, i])
        shuff_acc = r2_score(Y_test, rf.predict(X_t))
        scores[names[i]].append((acc-shuff_acc)/acc)
        
print("Features sorted by their score:")
print(sorted([(round(np.mean(score), 4), feat) for feat, score in scores.items()], reverse=True))

Features sorted by their score:
[(0.0, 'RTY Index'), (0.0, 'NDUEACWF Index'), (0.0, 'NDDUEAFE Index'), (0.0, 'NDDUE15 Index'), (0.0, 'MIMUEMRN Index'), (0.0, 'M1WOEV Index'), (0.0, 'IDCOT3TR Index'), (0.0, 'FTAW01 Index'), (0.0, 'BFU5TRUU Index'), (0.0, 'BCIW1A Index')]
